# XML Summary
This part summerizes key elements in a Mujoco XML model that is important for rigid-body robotic systems. Complete and detailed information can be found in the official website 
https://mujoco.readthedocs.io/en/latest/XMLreference.html



## Frame orientation:

The frame orientation is specified using at most one of these attributes. The quat attribute has a default value corresponding to the null rotation, while the others are initialized in the special undefined state. Thus if none of these attributes are specified by the user, the frame is not rotated.

- quat: real(4), “1 0 0 0”
  If the quaternion is known, this is the preferred was to specify the frame orientation because it does not involve conversions. Instead it is normalized to unit length and copied into mjModel during compilation. When a model is saved as MJCF, all frame orientations are expressed as quaternions using this attribute.

- axisangle: real(4), optional
  These are the quantities (x,y,z,a) mentioned above. The last number is the angle of rotation, in degrees or radians as specified by the angle attribute of compiler. The first three numbers determine a 3D vector which is the rotation axis. This vector is normalized to unit length during compilation, so the user can specify a vector of any non-zero length. Keep in mind that the rotation is right-handed; if the direction of the vector (x,y,z) is reversed this will result in the opposite rotation. Changing the sign of a can also be used to specify the opposite rotation.

- euler: real(3), optional
  Rotation angles around three coordinate axes. The sequence of axes around which these rotations are applied is determined by the eulerseq attribute of compiler and is the same for the entire model.

- xyaxes: real(6), optional
  The first 3 numbers are the X axis of the frame. The next 3 numbers are the Y axis of the frame, which is automatically made orthogonal to the X axis. The Z axis is then defined as the cross-product of the X and Y axes.

- zaxis: real(3), optional
  The Z axis of the frame. The compiler finds the minimal rotation that maps the vector (0,0,1) into the vector specified here. This determines the X and Y axes of the frame implicitly. This is useful for geoms with rotational symmetry around the Z axis, as well as lights – which are oriented along the Z axis of their frame.


## body
- need to specify: name (optional)
- pos, orientation (see frame orientation section) relative to parant coordinate system
- and important sub-element (geom, inertia, joint, etc)

### body/inertial:
- This element specifies the mass and inertial properties of the body. If this element is not included in a given body, the inertial properties are inferred from the geoms attached to the body. When a compiled MJCF model is saved, the XML writer saves the inertial properties explicitly using this element, even if they were inferred from geoms. The inertial frame is such that its center coincides with the center of mass of the body, and its axes coincide with the principal axes of inertia of the body. Thus the inertia matrix is diagonal in this frame.
- *pos*: position of inertial frame. 
- *orientation*: of the inertial frame
- *mass*: positive number required
- *diaginertia* (real(3)): diagonal entries of the inertial matrix;
- *fullinertia*: real(6): Full inertia matrix M. Since M is 3-by-3 and symmetric, it is specified using only 6 numbers in the following order: M(1,1), M(2,2), M(3,3), M(1,2), M(1,3), M(2,3). 

### body/joint
A joint creates motion degrees of freedom between the body where it is defined and the body’s parent. If no joints are defined, the body is welded to its parent. Joints cannot be defined in the world body. At runtime the positions and orientations of all joints defined in the model are stored in the vector mjData.qpos, in the order in which they appear in the kinematic tree. The linear and angular velocities are stored in the vector mjData.qvel. These two vectors have different dimensionality when free or ball joints are used, because such joints represent rotations as unit quaternions.
  - **name**, **class**: optional 
  - **type**: 
    - *free*: 6Dof joint. This joint type is only allowed in bodies that are children of the world body. Free joints do not have a position within the body frame. Instead the joint position is assumed to coincide with the center of the body frame. Thus at runtime the position and orientation data of the free joint correspond to the global position and orientation of the body frame. Free joints cannot have limits.
    - *slide*: a sliding or prismatic joint with one translational degree of freedom. Such joints are defined by a position and a sliding direction. For simulation purposes only the direction is needed; the joint position is used for rendering purposes.
    - *hinge*: The rotation takes place around a specified axis through a specified position. This is the most common type of joint and is therefore the default. 
  - **pos**: Position of the joint, specified in the frame of the body where the joint is defined. For free joints this attribute is ignored.
  - **axis**: This attribute specifies the axis of rotation for hinge joints and the direction of translation for slide joints.

### body/geom
- need to specify: name (optional), class (optional), pos, orientation (see frame orientation section), type, size (different meanings for different types), material (reference to materials defined in assets), rgba, among others. 
  
- **type**: (default: "sphere")
  - "box": The box type defines a box. Three size parameters are required, corresponding to the half-sizes of the box along the X, Y and Z axes of the geom’s frame. 
  - "cylinder": defines a cylinder, which requires two size parameters: the radius and half-height of the cylinder. The cylinder is oriented along the Z axis of the geom’s frame. 
  - "mesh": defines a mesh. The geom must reference the desired mesh asset with the mesh attribute. Note that mesh assets can also be referenced from other geom types, causing primitive shapes to be fitted; see below. The size is determined by the mesh asset and the geom size parameters are ignored.
- **fromto**: This attribute can only be used with capsule, box, cylinder and ellipsoid geoms. It provides an alternative specification of the geom length as well as the frame position and orientation. The six numbers are the 3D coordinates of one point followed by the 3D coordinates of another point. 
- **pos**: (x,y,z) position 
- orietation related: see frame orietation section
- **material**: 
- **rgba**: 


## asset
Assets are not in themselves model elements. Model elements can reference them, in which case the asset somehow changes the properties of the referencing element. One asset can be referenced by multiple model elements. Since the sole purpose of including an asset is to reference it, and referencing can only be done by name, every asset has a name (which may be inferred from a file name when applicable).

### asset/mesh
MuJoCo works with triangulated meshes. They can be loaded from binary STL files, OBJ files or MSH files. 

### asset/material
It can be referenced from skins, geoms, sites and tendons to set their appearance. Materials are useful for adjusting appearance properties beyond color.

## actuator
MuJoCo provides a flexible actuator model. All actuators are single-input-single-output (SISO). 
- The input to actuator $i$ a scalar control $u_i$ specified by the user. 
- The output is a scalar force $p_i$, which is mapped to joint coordinates by a vector of moment arms determined by the transmission. 
- An actuator can also have activation state $w_i$ with its own dynamics
- The control inputs for all actuators are stored in mjData.ctrl, the force outputs are stored in mjData.actuator_force, and the activation states (if any) are stored in mjData.act.
  
### actuator/general:
- joint/site: either specify a joint or a site for applying the force. 
- dyntype ("none"): internal dynamics type, including integrator, filter, filerexact, muscle, user... . The last case with "user" allows customize callback functions for actuator dynamics. 
- gaintype: The gain and bias together determine the output of the force generation mechanism, which is currently assumed to be affine. The general formula is: 
      **scalar_force = gain_term * (act or ctrl) + bias_term**
- ctrllimited: If true, the control input to this actuator is automatically clamped to ctrlrange at runtime
- forcelimited: If true, the force output of this actuator is automatically clamped to forcerange at runtime

### pre-defined actuator options: 
Below are actuator shortcuts discussed earlier. When a such shortcut is encountered, the parser creates a general actuator and sets its dynprm, gainprm and biasprm attributes to the internal defaults shown above, regardless of any default settings.

- actuator/motor: 

| Attribute   | Setting       | Attribute   | Setting       |
|-------------|---------------|-------------|---------------|
| dyntype     | none          | gaintype    | fixed         |
| dynprm      | 1 0 0        | gainprm     | 1 0 0        |
| biastype     | none          | biasprm     | 0 0 0        |

- actuator/position:
  This element creates a position servo with an optional first-order filter. The underlying general attributes are set as follows: 

| Attribute   | Setting                  | Attribute   | Setting            |
|-------------|--------------------------|-------------|---------------------|
| dyntype     | none or filterexact      | gaintype    | fixed               |
| dynprm      | timeconst 0 0            | gainprm     | kp 0 0              |
| biastype     | affine                  | biasprm     | 0 -kp -kv           |
  
- actuator/velocity
  This element creates a velocity servo. Note that in order create a PD controller, one has to define two actuators: a position servo and a velocity servo. This is because MuJoCo actuators are SISO while a PD controller takes two control inputs (reference position and reference velocity). When using this actuator, it is recommended to use the implicitfast or implicit integrators. The underlying general attributes are set as follows:

| Attribute   | Setting       | Attribute   | Setting        |
|-------------|---------------|-------------|-----------------|
| dyntype     | none          | gaintype    | fixed           |
| dynprm      | 1 0 0        | gainprm     | kv 0 0          |
| biastype     | affine        | biasprm     | 0 0 -kv         |

- actuator/damper
  

## Sensor
The outputs of all sensors are concatenated in the field mjData.sensordata which has size mjModel.nsensordata. This data is not used in any internal computations.

- Sipmle example: 
  > <sensor>
        <jointpos name="joint1_position_sensor" joint="joint1"/> 
        <jointvel name="joint1_velocity_sensor" joint="joint1"/> 
    </sensor>
- other attributes:
  - **force**: 3-axis force. The sensor outputs three numbers, which are the interaction force between a child and a parent body, expressed in the site frame defining the sensor. The convention is that the site is attached to the child body, and the force points from the child towards the parent.
    - note that *site* sub-element is required. 
  - **torque**:  a 3-axis torque similar to force
  - 